<a href="https://colab.research.google.com/github/seanjudelyons/RMAI_CandidatePack/blob/main/Solution_2_CommentClassification_Candidate_Lyons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers
!git clone https://github.com/seanjudelyons/RMAI_CandidatePack
%cd RMAI_CandidatePack/Comment Classification

     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 16.1MB/s 
     |████████████████████████████████| 890kB 41.4MB/s 
     |████████████████████████████████| 3.0MB 30.7MB/s 
     |████████████████████████████████| 1.1MB 54.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7.2-cp36-none-any.whl size=91190 sha256=a751d88805cae449abde77693d8ea30e62f12aab73e5b15ea9f651d51a361c4a
  Stored in directory: /root/.cache/pip/wheels/4b/8f/23/7f93e91176acc7c2d651b54fa9f01e3624a47904145c4d133e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6adfca2ed65485b348cd210dced90a260a26c4cec20759aa83808d8962f19f48
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
Cloning into 'RMAI_CandidatePack'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/1

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sentence_transformers import SentenceTransformer
import csv
import io
import os
fpath = os.getcwd(); fpath

'/content/RMAI_CandidatePack/Comment Classification'

In [3]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

with open(fpath + '/Delay Sample Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      label= float(row[1])
      data = row[0]
      training_sentences.append(data)
      training_labels.append(label)

with open(fpath + '/Delay Scoring Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      label= float(row[1])
      data = row[0]
      testing_sentences.append(data)
      testing_labels.append(label)

In [4]:
print(len(training_sentences))
print(len(training_labels))
print(len(testing_sentences))
print(len(testing_labels))
test_len=len(testing_labels)

2432
2432
608
608


In [5]:
print(training_sentences[0])

truck dog wet dirt lost traction


In [6]:
for i in range(test_len):
  training_sentences.append(testing_sentences[i])
  training_labels.append(testing_labels[i])

In [7]:
sentences= np.array(training_sentences)
labels= np.array(training_labels)
print(sentences.shape)
print(labels.shape)

(3040,)
(3040,)


In [8]:
print(sentences[3000])
print(labels[3000])

maintain focus wind back operations exceeding level operation safely supervised controlled
1.0


In [9]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:24<00:00, 16.7MB/s]


In [10]:
sentence_embeddings = model.encode(sentences)

In [11]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /content/glove.6B.100d.txt

--2020-10-08 15:27:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 2607:f8b0:400e:c07::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/content/glove.6B.100d.txt’

/content/glove.6B.1 100%[===================>] 331.04M   125MB/s    in 2.7s    

2020-10-08 15:27:48 (125 MB/s) - ‘/content/glove.6B.100d.txt’ saved [347116733/347116733]



In [12]:
y= "yes"
n= "no"
   
with open('/content/glove.6B.100d.txt') as f:
   
     for line in f:
      values = line.split();
      word = values[0];
      if y==word:
        coefs = np.asarray(values[1:], dtype='float32');
        yes_embedding = coefs;
      if n==word:
        coefs = np.asarray(values[1:], dtype='float32');
        no_embedding = coefs;

In [13]:
print(sentence_embeddings.shape)
print(yes_embedding.shape)
print(no_embedding.shape)

(3040, 768)
(100,)
(100,)


In [14]:
yn_vecs=[]

for i in labels:
  if i==1:
    yn_vecs.append(yes_embedding)
  else:
    yn_vecs.append(no_embedding)

add_embedding=np.array(yn_vecs)
print(add_embedding.shape)

(3040, 100)


In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
y= pad_sequences(add_embedding, padding='post', truncating='post', maxlen=768)
print(y)
print(y.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(3040, 768)


In [16]:
print(y.shape)
print(sentence_embeddings.shape)

(3040, 768)
(3040, 768)


In [17]:
model_1= keras.Sequential([keras.layers.Dense(units= 768, input_dim= 768,trainable=False)])

In [18]:
model_1.compile(optimizer='sgd', loss='mean_squared_error')

In [19]:
model_1.fit(sentence_embeddings, y, epochs=100)

Epoch 1/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 2/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 3/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 4/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 5/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 6/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 7/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 8/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 9/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 10/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 11/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 12/100
95/95 [==============================] - 0s 1ms/step - loss: 0.3475
Epoch 13/100
95/95 [=================

In [20]:
query = "Asbestos pipe removed on site." #Enter Query Hwew

In [21]:
queries = [query]
query_embeddings = model.encode(queries)


number_top_matches = 3 

In [22]:
import scipy
number_top_matches = 10 #@param {type: "number"}
print("Semantic Search Results")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: Asbestos pipe removed on site.

Top 5 most similar sentences in corpus:
remove asbestos pipe s3 (Cosine Score: 0.8426)
works started removing redundant asbestos water main abutment (Cosine Score: 0.8204)
material treated asbestos robson started removal material accordance standards (Cosine Score: 0.8165)
earthen ware pipe discovered scott st opposite newcastle station demolition works hoarding area (Cosine Score: 0.8024)
material thought asbestos uncovered demolition valley kitchens building (Cosine Score: 0.8022)
prior partial demolition newcastle station nlr construction contamination testing platform undertaken (Cosine Score: 0.7733)
asbestos near redundant cable (Cosine Score: 0.7708)
removing section concrete existing footpath gas leak occurred (Cosine Score: 0.7594)
removal asbestos water main overhead powerlines delaying excavation s3 abutment (Cosine Score: 0.7573)
extent contamination not known material test results confirm material indeed as

In [23]:
actual_sentences=[]
actual_labels=[]

with open(fpath + '/Delay Actual Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      label= row[1]
      data = row[0]
      actual_sentences.append(data)
      actual_labels.append(label)

In [24]:
print(len(actual_sentences))

225


In [25]:
vectored_sentences = model.encode(actual_sentences)

In [26]:
modelprediction= (model_1.predict([vectored_sentences]))

In [27]:
print(modelprediction.shape)

(225, 768)


In [28]:
weights=modelprediction

In [30]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for count, sent in enumerate(actual_sentences):
  embeddings = weights[count]
  a_l=actual_labels[count]
  tem="label:"+str(a_l)+" "+sent
  out_m.write(tem + "\n")
  out_v.write('\t'.join(str(x) for x in embeddings) + "\n")
out_v.close()
out_m.close()

In [31]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>